<a href="https://colab.research.google.com/github/adammoss/MLiS2/blob/master/workshops/workshop5/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In lectures we hand-coded the BPTT algorithm to train an RNN language model to predict the next word in a sentence.

Using the same training corpus, train a many-to-many LSTM model using TF2 to perform the same task, and compare your results against a vanilla RNN.

In this example we concatenate all the sentences into a single vector to make it easier to feed into the TF2 dataset API. We therefore only have a single stop word between sentences.

The downside of this approach is that sentences in different reviews will have different context, so ideally we would treat different reviews seperately and pad inputs where necessary. 

Tensorflow has a similar character level RNN (not at the word level) here to help you: https://www.tensorflow.org/tutorials/text/text_generation


In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import csv
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime
import os

import matplotlib.pyplot as plt
%matplotlib inline

In [132]:
print(tf.__version__)

2.1.0


Download NLTK data

In [0]:
%%capture
nltk.download("book")

Upload imdb_sentences.txt file (or another file containing a list of sentences if you wish)

In [0]:
if not os.path.isfile('imdb_sentences.txt'):
  from google.colab import files
  uploaded = files.upload()

Add sentence start and end tags, convert to lower case and strip newlines

In [0]:
sentence_start_token = "SENTENCE_STOP"

In [0]:
with open('imdb_sentences.txt', 'r') as f:
  sentences = f.readlines()
sentences = ["%s %s" % (sentence_start_token, x.lstrip().rstrip('.\n').lower()) for x in sentences]

In [137]:
print("Parsed %d sentences." % (len(sentences)))
for i in range(0, 10):
  print("Example: %s" % sentences[i])

Parsed 12188 sentences.
Example: SENTENCE_STOP story of a man who has unnatural feelings for a pig
Example: SENTENCE_STOP starts out with a opening scene that is a terrific example of absurd comedy
Example: SENTENCE_STOP a formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers
Example: SENTENCE_STOP unfortunately it stays absurd the whole time with no general narrative eventually making it just too off putting
Example: SENTENCE_STOP even those from the era should be turned off
Example: SENTENCE_STOP the cryptic dialogue would make shakespeare seem easy to a third grader
Example: SENTENCE_STOP on a technical level it's better than you might think with some good cinematography by future great vilmos zsigmond
Example: SENTENCE_STOP future stars sally kirkland and frederic forrest can be seen briefly
Example: SENTENCE_STOP airport '77 starts as a brand new luxury 747 plane is loaded up with valuable paintings & such belonging to rich business

Tokenize the sentences into words

In [0]:
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

In [139]:
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print("Found %d unique words tokens." % len(word_freq.items()))

Found 18153 unique words tokens.


In [0]:
vocab_size = 1000
unknown_token = 'UNKNOWN_TOKEN'

In [0]:
vocab = word_freq.most_common(vocab_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
index_to_word = np.array(index_to_word)
word_to_index = dict([(w,i) for i, w in enumerate(index_to_word)])

Replace all words not in our vocabulary with the unknown token and discard sentences under min / over max number of words

In [0]:
purged_sentences = []
for i, sent in enumerate(tokenized_sentences):
  if len(sent) >= min_sentence_length:
    purged_sentences.append([w if w in word_to_index else unknown_token for w in sent])


Flatten sentences

In [0]:
text = [word for sent in purged_sentences for word in sent]
    

Convert to integer representations

In [0]:
text_as_int = np.array([word_to_index[w] for w in text])

Set maximum length sentence we want for a single input in characters

In [0]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

Create the dataset

In [0]:
dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [149]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(' '.join(index_to_word[input_example.numpy()])))
  print ('Target data:', repr(' '.join(index_to_word[target_example.numpy()])))

Input data:  "SENTENCE_STOP story of a man who has UNKNOWN_TOKEN UNKNOWN_TOKEN for a UNKNOWN_TOKEN SENTENCE_STOP starts out with a opening scene that is a UNKNOWN_TOKEN example of absurd comedy SENTENCE_STOP a UNKNOWN_TOKEN UNKNOWN_TOKEN audience is turned into an UNKNOWN_TOKEN , UNKNOWN_TOKEN UNKNOWN_TOKEN by the crazy UNKNOWN_TOKEN of it 's UNKNOWN_TOKEN SENTENCE_STOP unfortunately it UNKNOWN_TOKEN absurd the whole time with no general UNKNOWN_TOKEN eventually making it just too off UNKNOWN_TOKEN SENTENCE_STOP even those from the UNKNOWN_TOKEN should be turned off SENTENCE_STOP the UNKNOWN_TOKEN dialogue would make UNKNOWN_TOKEN seem UNKNOWN_TOKEN to a third UNKNOWN_TOKEN SENTENCE_STOP on a UNKNOWN_TOKEN level it 's better than you"
Target data: "story of a man who has UNKNOWN_TOKEN UNKNOWN_TOKEN for a UNKNOWN_TOKEN SENTENCE_STOP starts out with a opening scene that is a UNKNOWN_TOKEN example of absurd comedy SENTENCE_STOP a UNKNOWN_TOKEN UNKNOWN_TOKEN audience is turned into an UNKN

In [0]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

**Now code and train your RNN...**